In [2]:
import math
import pandas as pd
import os

In [3]:
# Configurations
pd.set_option('display.max_columns', None)

In [5]:
# Get CSVs and store them in a dictionary

pbp = {}
NGS_pbp = {}
years = [18, 19, 20, 21, 22, 23]
for x in years:
    pbp[x] = pd.read_csv('data/raw/pbp/play_by_play_20' + str(x) + '.csv')
    NGS_pbp[x] =  pd.read_csv('data/raw/pbp_participation/pbp_participation_20' + str(x) + '.csv')

# print(pbp[18].head())

print(NGS_pbp[18].head())

/var/folders/z2/rm0rvjws51g2b4p7_0ns_31w0000gp/T/ipykernel_14131/1155609648.py:7: DtypeWarning: Columns (40,41,44,45,49,50,51,52,179,180,182,183,187,188,191,192,193,194,195,196,197,198,199,200,203,204,205,206,207,208,211,212,213,214,215,216,217,218,219,220,222,224,226,227,228,229,230,231,232,240,241,242,246,247,248,249,250,251,252,253,254,255,256,258,259,260,262,263,264,265,266,267,268,269,270,272,273,274,277,278,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp[x] = pd.read_csv('data/raw/pbp/play_by_play_20' + str(x) + '.csv')
/var/folders/z2/rm0rvjws51g2b4p7_0ns_31w0000gp/T/ipykernel_14131/1155609648.py:7: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,218,219,220,233,234,235,236,237,238,248,249,253,254,255,260,262,263,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp[x] = pd.read_csv('data/raw/pbp/play_by_play_20' + str(x) + '.csv')
/var/folders/z2/rm0rvjws51g2b4p

  nflverse_game_id  old_game_id  play_id possession_team offense_formation  \
0  2018_01_ATL_PHI   2018090600        1             NaN               NaN   
1  2018_01_ATL_PHI   2018090600       37             PHI               NaN   
2  2018_01_ATL_PHI   2018090600       52             ATL               NaN   
3  2018_01_ATL_PHI   2018090600       75             ATL            I_FORM   
4  2018_01_ATL_PHI   2018090600      104             ATL            I_FORM   

  offense_personnel  defenders_in_box defense_personnel  \
0               NaN               NaN               NaN   
1               NaN               NaN               NaN   
2  2 RB, 1 TE, 2 WR               NaN  4 DL, 2 LB, 5 DB   
3  2 RB, 1 TE, 2 WR               7.0  4 DL, 2 LB, 5 DB   
4  2 RB, 1 TE, 2 WR               7.0  4 DL, 2 LB, 5 DB   

   number_of_pass_rushers                                    players_on_play  \
0                     NaN                                                NaN   
1               

# Drop Columns

Both datasets have substantially more columns we could ever need, particularly `pbp`. Most columns contain extraneous and niche information. I'm going to drop as few columns as possible since it's easier to drop than re-add, but the process of re-inserting droped columns is not very difficult.

Note: we should keep `play_id` and `game_id` since they're unique values that we can use to INNER JOIN these two dataframes together into one.

Also note: We may want to drop rows *then* columns, as we may want to drop niche plays (e.g. safeties, laterals, punt return TDs, turnovers, etc.) then drop their corresponding columns.

## Drop Columns for pbp_participation

Drop columns that contain player IDs number of players on offense/defense (it's almost always 11, who knew).

In [15]:
# Choose columns to drop

ngs_dropColumns = ["players_on_play", 
               "offense_players", 
               "defense_players",
               "n_offense",
               "n_defense"]

In [18]:
# Execute column drops for each dataframe in pbp_particpation

for x in years:
    NGS_pbp[x] = NGS_pbp[x].drop(columns = ngs_dropColumns)

In [19]:
# See if columns were dropped

NGS_pbp[18].head()

,nflverse_game_id,old_game_id,play_id,possession_team,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,ngs_air_yards,time_to_throw,was_pressure,route,defense_man_zone_type,defense_coverage_type
0,2018_01_ATL_PHI,2018090600,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018_01_ATL_PHI,2018090600,37,PHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018_01_ATL_PHI,2018090600,52,ATL,NaN,"2 RB, 1 TE, 2 WR",NaN,"4 DL, 2 LB, 5 DB",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018_01_ATL_PHI,2018090600,75,ATL,I_FORM,"2 RB, 1 TE, 2 WR",7.0,"4 DL, 2 LB, 5 DB",4.0,8.89,2.502,False,HITCH,ZONE_COVERAGE,COVER_3
4,2018_01_ATL_PHI,2018090600,104,ATL,I_FORM,"2 RB, 1 TE, 2 WR",7.0,"4 DL, 2 LB, 5 DB",NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Drop Columns for pbp

Unfortunately, since there are so 300+ columns, this is obviously a much more difficult process. I'll organize which columns to drop by "theme" as I think this will be the clearest and easiest way to do so.

In [26]:
# Choose columns to drop based on "theme"

gameInfo = ["old_game_id", "week", "game_date", "season"] # consider dropping team/posession info

metaData = ["order_sequence", "start_time", "time_of_day", "stadium", 
            "weather", "nfl_api_id"]

timeLeft = ["quarter_seconds_remaining", "half_seconds_remaining", 
            "game_seconds_remaining", "game_half", "quarter_end", "drive"] 

situational = ["goal_to_go", "yrdin", "ydsnet"] 

# TODO: Consider keeping "play_type" and dropping rows where it's "kickoff" instead
#
# TODO: Same with "qb_kneel" or "qb_spike", consider dropping rows where it's 1 (i.e. true) 
# instead of dropping altogether
playInfo = ["desc", "shotgun", "qb_dropback", "qb_kneel", "qb_spike", 
            "incomplete_pass", "complete_pass"]

playType = ["extra_point_attempt",	"two_point_attempt", "field_goal_attempt",	
            "kickoff_attempt",	"punt_attempt", "rush_attempt", "pass_attempt"]

kicking_and_XP = ["field_goal_result", "kick_distance", 
                  "extra_point_result", "two_point_conv_result",
                 "extra_point_prob", "two_point_conversion_prob", "home_opening_kickoff"]

timeouts = ["home_timeouts_remaining",	"away_timeouts_remaining",
            "timeout", "timeout_team", "posteam_timeouts_remaining",
            "defteam_timeouts_remaining"]

tdInfo = ["td_team", "td_player_name",	"td_player_id"]

scoreDiff = ["posteam_score_post", "defteam_score_post", "score_differential_post"]

ptsTypeProb = ["opp_fg_prob", "opp_safety_prob", "opp_td_prob", 
               "fg_prob", "safety_prob", "td_prob", "extra_point_prob", 
               "two_point_conversion_prob"]

epa = ["total_home_epa", "total_away_epa", "total_home_rush_epa", 
       "total_away_rush_epa", "total_home_pass_epa", "total_away_pass_epa", 
       "air_epa", "yac_epa", "comp_air_epa", "comp_yac_epa", 
       "total_home_comp_air_epa", "total_away_comp_air_epa", 
       "total_home_comp_yac_epa", "total_away_comp_yac_epa", 
       "total_home_raw_air_epa", "total_away_raw_air_epa", 
       "total_home_raw_yac_epa", "total_away_raw_yac_epa"]

wpa = ["vegas_wpa", "vegas_home_wpa", "home_wp_post", "away_wp_post", 
       "vegas_wp", "vegas_home_wp", "total_home_rush_wpa", "total_away_rush_wpa", 
       "total_home_pass_wpa", "total_away_pass_wpa", "air_wpa", "yac_wpa", 
       "comp_air_wpa", "comp_yac_wpa", "total_home_comp_air_wpa", 
       "total_away_comp_air_wpa", "total_home_comp_yac_wpa", 
       "total_away_comp_yac_wpa", "total_home_raw_air_wpa", "total_away_raw_air_wpa", 
       "total_home_raw_yac_wpa", "total_away_raw_yac_wpa"]

downInfo = ["first_down_pass", "first_down_penalty", "third_down_converted", 
            "third_down_failed", "fourth_down_converted", "fourth_down_failed"]

punt_and_kickoff = ["punt_blocked", "touchback" "punt_inside_twenty", "punt_in_endzone", 
                    "punt_out_of_bounds", "punt_downed", "punt_fair_catch", 
                    "kickoff_inside_twenty", "kickoff_in_endzone", "kickoff_out_of_bounds", 
                    "kickoff_downed", "kickoff_fair_catch", "own_kickoff_recovery"
                    ,"own_kickoff_recovery_td"]

turnover = ["interception", "fumble_forced", "fumble_not_forced", "fumble_out_of_bounds",
           "fumble_lost", "fumble"]

defensive = ["solo_tackle", "safety", "qb_hit", "sack", "tackled_for_loss", "assist_tackle"]

touchdown = ["touchdown", "pass_touchdown", "rush_touchdown", "return_touchdown"]

lateral = ["lateral_reception", "lateral_rush", "lateral_return", "lateral_recovery",
          "lateral_receiver_player_id", "lateral_receiver_player_name", 
           "lateral_receiving_yards", "lateral_rusher_player_id", 
           "lateral_rusher_player_name", "lateral_rushing_yards", 
           "lateral_sack_player_id", "lateral_sack_player_name",
          "lateral_interception_player_name","lateral_punt_returner_player_id",
           "lateral_punt_returner_player_name"]

offPlayerID = ["passer_player_id", "passer_player_name", "passing_yards", 
               "receiver_player_id", "receiver_player_name", "receiving_yards", 
               "rusher_player_id", "rusher_player_name", "rushing_yards"]

puntPlayerID = ["punt_returner_player_id", "punt_returner_player_name",
               "punter_player_id", "punter_player_name"]

kickingPlayerID = ["kickoff_returner_player_name",	"kickoff_returner_player_id",
                   "kicker_player_name", "kicker_player_id",
                   "own_kickoff_recovery_player_id", "own_kickoff_recovery_player_name", 
                   "blocked_player_id", "blocked_player_name", "return_team", "return_yards"]

defensePlayerID = ["tackle_for_loss_1_player_id", "tackle_for_loss_1_player_name", 
                   "tackle_for_loss_2_player_id", "tackle_for_loss_2_player_name", 
                   "qb_hit_1_player_id", "qb_hit_1_player_name", "qb_hit_2_player_id", 
                   "qb_hit_2_player_name", "forced_fumble_player_1_team", 
                   "forced_fumble_player_1_player_id", "forced_fumble_player_1_player_name", 
                   "forced_fumble_player_2_team", "forced_fumble_player_2_player_id", 
                   "forced_fumble_player_2_player_name", "solo_tackle_1_team", "solo_tackle_2_team", 
                   "solo_tackle_1_player_id", "solo_tackle_2_player_id", "solo_tackle_1_player_name",
                   "solo_tackle_2_player_name", "assist_tackle_1_player_id", "assist_tackle_1_player_name", 
                   "assist_tackle_1_team", "assist_tackle_2_player_id", "assist_tackle_2_player_name", 
                   "assist_tackle_2_team", "assist_tackle_3_player_id", "assist_tackle_3_player_name", 
                   "assist_tackle_3_team", "assist_tackle_4_player_id", "assist_tackle_4_player_name", 
                   "assist_tackle_4_team", "tackle_with_assist", "tackle_with_assist_1_player_id", 
                   "tackle_with_assist_1_player_name", "tackle_with_assist_1_team", 
                   "tackle_with_assist_2_player_id", "tackle_with_assist_2_player_name",
                   "tackle_with_assist_2_team", "pass_defense_1_player_id", "pass_defense_1_player_name", 
                   "pass_defense_2_player_id", "pass_defense_2_player_name", "fumbled_1_team", 
                   "fumbled_1_player_id", "fumbled_1_player_name", "fumbled_2_player_id", 
                   "fumbled_2_player_name", "fumbled_2_team", "fumble_recovery_1_team", 
                   "fumble_recovery_1_yards", "fumble_recovery_1_player_id", 
                   "fumble_recovery_1_player_name", "fumble_recovery_2_team", "fumble_recovery_2_yards", 
                   "fumble_recovery_2_player_id", "fumble_recovery_2_player_name", "sack_player_id", 
                   "sack_player_name", "half_sack_1_player_id", "half_sack_1_player_name", 
                   "half_sack_2_player_id", "half_sack_2_player_name"]

penaltyPlayerId = ["penalty_player_id",	"penalty_player_name",	"penalty_yards"]

ref = ["replay_or_challenge", "replay_or_challenge_result",	"penalty_type"]

safety = ["defensive_two_point_attempt", "defensive_two_point_conv", 
          "defensive_extra_point_attempt", "defensive_extra_point_conv", 
          "safety_player_name", "safety_player_id"]

cpoe = ["cp", "cpoe"]

series = ["series", "series_success", "series_result", "order_sequence"]

playClock = ["play_clock", "play_deleted"]

stPlay = ["special_teams_play",	"st_play_type"]

drive = ["end_clock_time", "end_yard_line", "fixed_drive", "fixed_drive_result", 
         "drive_real_start_time", "drive_play_count", "drive_time_of_possession", 
         "drive_first_downs", "drive_inside20", "drive_ended_with_score", 
         "drive_quarter_start", "drive_quarter_end", "drive_yards_penalized", 
         "drive_start_transition", "drive_end_transition", "drive_game_clock_start", 
         "drive_game_clock_end", "drive_start_yard_line", "drive_end_yard_line", 
         "drive_play_id_started", "drive_play_id_ended"]

gameScoreData = ["away_score", "home_score", "location", "result", "total", 
                 "spread_line", "total_line", "div_game"]

weather_stadium_conditions = ["roof", "surface", "temp", "wind", 
                              "stadium_id", "game_stadium"]

coaches = ["home_coach", "away_coach"]


abortedPlay = ["aborted_play", "success"]

offJerseys = ["passer", "passer_jersey_number", "rusher", 
                 "rusher_jersey_number", "receiver", "receiver_jersey_number",
             "passer_id", "rusher_id", "receiver_id", "name", "jersey_number", "id"]

playEncoding = ["pass", "rush", "first_down", "special", "play"]

fantasy = ["fantasy_player_name", "fantasy_player_id", "fantasy", "fantasy_id"]

exp_epa = ["qb_epa", "xyac_epa", "xyac_mean_yardage", "xyac_median_yardage", 
           "xyac_success", "xyac_fd", "xpass", "pass_oe"]

# Should this be dropped? How would this affect EPA?
# penalty = ["penalty"]

In [30]:

# Dictionary mapping list names to their respective lists
columnTypes = {
    "playInfo": playInfo,
    "playType": playType,
    "kicking_and_XP": kicking_and_XP,
    "timeouts": timeouts,
    "tdInfo": tdInfo,
    "scoreDiff": scoreDiff,
    "ptsTypeProb": ptsTypeProb,
    "epa": epa,
    "wpa": wpa,
    "downInfo": downInfo,
    "punt_and_kickoff": punt_and_kickoff,
    "turnover": turnover,
    "defensive": defensive,
    "touchdown": touchdown,
    "lateral": lateral,
    "offPlayerID": offPlayerID,
    "puntPlayerID": puntPlayerID,
    "kickingPlayerID": kickingPlayerID,
    "defensePlayerID": defensePlayerID,
    "penaltyPlayerId": penaltyPlayerId,
    "ref": ref,
    "safety": safety,
    "cpoe": cpoe,
    "series": series,
    "playClock": playClock,
    "stPlay": stPlay,
    "drive": drive,
    "gameScoreData": gameScoreData,
    "weather_stadium_conditions": weather_stadium_conditions,
    "coaches": coaches,
    "abortedPlay": abortedPlay,
    "offJerseys": offJerseys,
    "playEncoding": playEncoding,
    "fantasy": fantasy,
    "exp_epa": exp_epa,
}


In [31]:
for x in years:
    for c in columnTypes:
        pbp[x] = pbp[x].drop(columns = c)

KeyError: "['playInfo'] not found in axis"